Name AI
=======

Name AI is an machine learning application that can come up with brandable domain names.

How it works
============

For more understanding how it works, please refer to my blogpost: [Let AI come up with your next domain name!](https://codebuffet.co/2017/03/31/let-ai-come-up-with-your-next-domain-name/)

---

Copyright (C) 2017 Peter Willemsen

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
import os
os.environ['KERAS_BACKEND'] = 'theano'
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Dropout
from keras.layers import GRU
from keras.regularizers import l2, activity_l2
from keras.optimizers import RMSprop
from IPython.display import clear_output
import numpy as np
import math
from datetime import datetime
import time
import random
import sys
import string
import h5py

Using Theano backend.


In [2]:
np.random.seed()
random.seed()

Create character indices, so that each character is linked to a number.

In [3]:
mask = '_'
chars = None
with open('./domains.txt') as handle:
    text = handle.read().lower()[:-1]
    chars = sorted(list(set(text + mask)))
    print 'corpus length:', len(text)
    
weights_path = './model.hdf5'
print 'total chars:', len(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 544527
total chars: 29


In [4]:
def pad_name(name, max_length, skip = 0):
    padded_name = mask * (max_length - len(name) - skip)
    padded_name += name
    
    return padded_name

We count the longest domain name in the corpus. This length will be used to scale the rest of the data, so that no domain will be cut off. The rest will be left-padded with '_'.

In [5]:
maxlen = 3
domains = []
next_chars = []
maxlen2 = 0
with open('./domains.txt', 'r') as handle:
    for line in handle:
        line = line.strip()
        maxlen2 = max(maxlen2, len(line))

Put the data in a time-series window of 3 steps. A window of 3 characters means that the network will keep in mind the past 3 characters, while predicting the next single character.

In [6]:
step = 1
batch_size = maxlen2 * 100
with open('./domains.txt', 'r') as handle:
    padded_names = ""
    for line in handle:
        line = line.strip()                    
        padded_names += pad_name(line, maxlen2)

for i in range(0, len(padded_names) - maxlen, step):
    domains.append(padded_names[i: i + maxlen])
    next_chars.append(padded_names[i + maxlen])

for i in range(0, maxlen2*2):
    print "%s -> %s" % (domains[i], next_chars[i])

print 'nb sequences:', len(domains), len(next_chars)
print "batch size: %d maxlen2: %d" % (batch_size, maxlen2)

___ -> _
___ -> _
___ -> c
__c -> a
_ca -> s
cas -> h
ash -> c
shc -> o
hco -> v
cov -> e
ove -> r
ver -> a
era -> g
rag -> e
age -> .
ge. -> c
e.c -> o
.co -> m
com -> _
om_ -> _
m__ -> _
___ -> _
___ -> _
___ -> _
___ -> _
___ -> d
__d -> u
_du -> a
dua -> l
ual -> o
alo -> p
lop -> t
opt -> i
pti -> c
tic -> s
ics -> .
cs. -> c
s.c -> o
.co -> m
com -> _
om_ -> _
m__ -> _
nb sequences: 918516 918516
batch size: 2100 maxlen2: 21


Vectorization - convert the time-series window from a list of characters to a list of numbers (the character embeddings)

In [7]:
X = np.zeros((len(domains), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(domains), len(chars)), dtype=np.bool)
for i, domain in enumerate(domains):
    for t, char in enumerate(domain):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Our model consists of a single GRU layer with 128 units (which is quite small). My goal was that Name AI would learn the linguistics of what makes a name brandable, in which case such a small model is a perfect fit. 

This model has a small dropout to make sure that we don't jolt our loss down to near-zero.

In [8]:
model = Sequential()
model.add(GRU(128, dropout_W=0.015, dropout_U=0.015, input_shape=(maxlen, len(chars)), return_sequences=False, stateful=False))

#for a hidden layer, uncomment this one
#model.add(GRU(128, dropout_W=0.015, dropout_U=0.015, return_sequences=False, stateful=False))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [9]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
if os.path.exists(weights_path):
    print "loading existing model.."
    model.load_weights(weights_path)

loading existing model..


In [11]:
def generate_alphabet_names():
    seeds = string.ascii_lowercase
    generate_names_amount = len(seeds)
    diversity = random.uniform(0.05, 0.5)

    print "Name AI by Peter Willemsen <peter@codebuffet.co>\nCreating %d names with diversity %f" % (generate_names_amount, diversity)
    for i in range(0, generate_names_amount):
        seed = pad_name(seeds[i], maxlen)
        sentence = seed
        generated = seed
        domains = generated

        for i in range(maxlen2 * 1):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            domains += next_char
        print domains.replace("_", "")

Train the model, output generated text after each iteration. We make sure we don't shuffle our data, otherwise the order of the 3-step window will be broken.

In [12]:
iteration = 0
while True:
    print 'Iteration', iteration
    model.fit(X, y, batch_size=batch_size, shuffle=False, nb_epoch=1, verbose=0)
    model.save_weights(weights_path, overwrite=True)
    clear_output()    
    generate_alphabet_names()      
        
    iteration += 1

Name AI by Peter Willemsen <peter@codebuffet.co>
Creating 26 names with diversity 0.396858
advino.com
beati.com
collio.com
dero.com
exa.com
finia.com
gonica.com
healthsy.com
ince.com
jolo.com
kindixity.com
lumerco.com
mobusia.com
nerce.com
ona.com
poptic.com
que.com
rexa.com
socity.com
telo.com
upsity.com
vero.com
webit.com
xoo.com
yamo.com
zen.com
Iteration 1


KeyboardInterrupt: 

In [ ]:
generate_alphabet_names()